In [ ]:
# Biblioteca para trabalhar com expressões regulares na análise léxica
import re

# Biblioteca para criar classes de dados de forma mais limpa
from dataclasses import dataclass, field 

# Tipos para anotação de tipos em Python, melhorando a legibilidade do código
from typing import List, Optional, Tuple

**Tokens e Lexer**

In [ ]:
# Define os padrões de todos os tokens que nosso analisador léxico pode reconhecer
# Cada tupla contém o nome do token e sua expressão regular correspondente
TOKEN_SPEC = [
    ("COMMENT",      r"//[^\n]*"),      # Comentários de linha que começam com //
    ("WS",           r"\s+"),           # Espaços em branco (espaços, tabs, quebras de linha)
    ("ANDAND",       r"&&"),            # Operador lógico AND
    ("OROR",         r"\|\|"),          # Operador lógico OR
    ("EQEQ",         r"=="),            # Operador de igualdade
    ("NEQ",          r"!="),            # Operador de diferença
    ("LE",           r"<="),            # Menor ou igual
    ("GE",           r">="),            # Maior ou igual
    ("ASSIGN",       r"="),             # Operador de atribuição
    ("LT",           r"<"),             # Menor que
    ("GT",           r">"),             # Maior que
    ("PLUS",         r"\+"),            # Soma
    ("MINUS",        r"-"),             # Subtração
    ("STAR",         r"\*"),            # Multiplicação
    ("SLASH",        r"/"),             # Divisão
    ("PERCENT",      r"%"),             # Módulo (resto da divisão)
    ("BANG",         r"!"),             # Negação lógica
    ("LPAREN",       r"\("),            # Parêntese esquerdo
    ("RPAREN",       r"\)"),            # Parêntese direito
    ("LBRACE",       r"\{"),            # Chave esquerda
    ("RBRACE",       r"\}"),            # Chave direita
    ("SEMI",         r";"),             # Ponto e vírgula
    ("COMMA",        r","),             # Vírgula
    ("NUMBER",       r"\d+"),           # Números inteiros
    ("IDENT",        r"[A-Za-z_][A-Za-z0-9_]*"), # Identificadores (nomes de variáveis, funções, etc.)
]

# Palavras reservadas da linguagem que têm significado especial
# Quando encontramos um identificador que está neste dicionário, o tratamos como palavra-chave
KEYWORDS = {
    "int": "INT",       # Tipo inteiro
    "bool": "BOOL",     # Tipo booleano
    "true": "TRUE",     # Valor verdadeiro
    "false": "FALSE",   # Valor falso
    "if": "IF",         # Condicional se
    "else": "ELSE",     # Condicional senão
    "while": "WHILE",   # Loop enquanto
}

# Compila todas as expressões regulares dos tokens em uma única expressão grande
# Isso permite que possamos verificar todos os padrões de uma vez só
MASTER_RE = re.compile("|".join(f"(?P<{name}>{pattern})" for name, pattern in TOKEN_SPEC))

@dataclass
class Token:
    """Representa um token encontrado no código fonte"""
    type: str      # Tipo do token (NUMBER, IDENT, IF, etc.)
    lexeme: str    # O texto original que formou este token
    line: int      # Linha onde o token foi encontrado
    col: int       # Coluna onde o token começou

class Lexer:
    """
    Analisador léxico que converte texto em uma sequência de tokens.
    Produz:
      - lista de tokens encontrados
      - 'tabela de lexemas e tokens' (é basicamente a mesma lista com metadados)
      - cadeia de tokens (string compacta dos tipos)
    Ignora espaços em branco e comentários. Reporta erro léxico com posição exata.
    """
    def __init__(self, source: str):
        self.source = source              # Código fonte a ser analisado
        self.tokens: List[Token] = []     # Lista onde armazenamos os tokens encontrados

    def tokenize(self) -> List[Token]:
        """Percorre o código fonte e extrai todos os tokens válidos"""
        line = 1                         # Linha atual (começa em 1)
        line_start_idx = 0               # Índice onde a linha atual começou
        i = 0                            # Posição atual no código fonte
        s = self.source                  # Referência local para o código fonte
        
        while i < len(s):                # Enquanto não chegamos ao fim do código
            m = MASTER_RE.match(s, i)    # Tenta fazer match de qualquer token na posição atual
            
            if not m:
                # Se não conseguiu reconhecer nenhum token, temos um erro léxico
                ch = s[i]                
                col = i - line_start_idx + 1  
                raise SyntaxError(f"[Lex] Caractere inesperado '{ch}' em {line}:{col}") 
            
            # Extraímos informações sobre o token encontrado
            kind = m.lastgroup           # Tipo do token (qual grupo da regex fez match)
            lexeme = m.group(kind)       # Texto que formou o token
            start = i                    # Posição onde o token começou
            end = m.end()                # Posição onde o token terminou
            newlines = lexeme.count("\n") # Quantas quebras de linha há no token
            
            # Se for espaço em branco ou comentário, ignoramos mas atualizamos a posição
            if kind == "WS" or kind == "COMMENT":
                i = end                  
                if newlines:
                    line += newlines     # Atualiza número da linha
                    last_nl = s.rfind("\n", start, end) 
                    line_start_idx = last_nl + 1        # Atualiza onde a linha atual começou
                continue                 

            # Verifica se o identificador é uma palavra-chave
            if kind == "IDENT" and lexeme in KEYWORDS:
                tok_type = KEYWORDS[lexeme]  # Usa o tipo da palavra-chave
            else:
                tok_type = kind              # Usa o tipo normal do token

            # Calcula a coluna e adiciona o token à lista
            col = start - line_start_idx + 1 
            self.tokens.append(Token(tok_type, lexeme, line, col)) 
            
            # Avança para a próxima posição
            i = end                         
            if newlines:
                line += newlines            
                last_nl = s.rfind("\n", start, end) 
                line_start_idx = last_nl + 1        

        # Adiciona token especial EOF (End Of File) para marcar o fim
        self.tokens.append(Token("EOF", "", line, (i - line_start_idx + 1))) 
        return self.tokens                

    def token_chain(self) -> str:
        """Retorna uma string com os tipos dos tokens separados por espaço (útil para debug)"""
        return " ".join(tok.type for tok in self.tokens if tok.type != "EOF")

**PARSER (Recursive Descent)**

In [ ]:
# Exceção personalizada para erros de análise sintática
class ParseError(Exception):  
    pass

class Parser:
    """
    Analisador sintático LL(1) que verifica se a sequência de tokens
    segue as regras gramaticais da nossa mini linguagem.
    
    Usa a técnica de descida recursiva, onde cada regra da gramática
    vira um método que chama outros métodos conforme necessário.
    """
    def __init__(self, tokens: List[Token]):  
        self.tokens = tokens  # Lista de tokens para analisar
        self.pos = 0          # Posição atual na lista de tokens

    def peek(self) -> Token:  
        """Olha o token atual sem consumi-lo"""
        return self.tokens[self.pos]

    def advance(self) -> Token:  
        """Consome o token atual e avança para o próximo"""
        tok = self.tokens[self.pos]
        self.pos += 1
        return tok

    def match(self, *types) -> bool:  
        """Verifica se o token atual é de um dos tipos esperados e o consome se for"""
        if self.peek().type in types:
            self.advance()
            return True
        return False

    def expect(self, ttype: str, msg: str):  
        """Exige que o token atual seja de um tipo específico, senão gera erro"""
        if self.peek().type == ttype:
            return self.advance()
        t = self.peek()
        raise ParseError(f"{msg} (encontrado {t.type} '{t.lexeme}' em {t.line}:{t.col})")

    def expect_ident_lexeme(self, name: str, msg: str):
        """Exige um identificador com um texto específico (exemplo: 'main')"""
        t = self.peek()
        if t.type == "IDENT" and t.lexeme == name:
            return self.advance()
        raise ParseError(f"{msg} (encontrado {t.type} '{t.lexeme}' em {t.line}:{t.col})")

    def look_type(self, k: int) -> str:
        """Olha o tipo do token que está k posições à frente (1 = próximo token)"""
        idx = self.pos + k
        if idx < len(self.tokens):
            return self.tokens[idx].type
        return "EOF"

    def parse(self):  
        """Método principal que inicia a análise sintática"""
        self.program()  # Analisa o programa completo
        self.expect("EOF", "Esperado fim do arquivo")  # Garante que chegamos ao final
        return True

    def program(self):  
        """Programa deve começar com 'main() { ... }'"""
        self.expect_ident_lexeme("main", "Esperado identificador 'main'")
        self.expect("LPAREN", "Esperado '(' após 'main'")
        self.expect("RPAREN", "Esperado ')' após '('")
        self.block()  # Analisa o bloco principal

    def block(self):  
        """Bloco: '{' declarações* comandos* '}'"""
        self.expect("LBRACE", "Esperado '{' para iniciar bloco")
        
        # Primeiro vêm todas as declarações de variáveis
        while self.peek().type in ("INT", "BOOL"):  
            self.decl()
        
        # Depois vêm os comandos
        while self.peek().type in ("IF", "WHILE", "IDENT", "LBRACE"):
            self.stmt()
            
        self.expect("RBRACE", "Esperado '}' para fechar bloco")

    def decl(self):  
        """Declaração de variável: tipo identificador [= expressão] ;"""
        if not self.match("INT", "BOOL"):
            t = self.peek()
            raise ParseError(f"Esperado tipo (int/bool), encontrado {t.type} em {t.line}:{t.col}")
        
        self.expect("IDENT", "Esperado identificador na declaração")
        
        # Inicialização é opcional
        if self.match("ASSIGN"):
            self.expr()  # Analisa a expressão de inicialização
            
        self.expect("SEMI", "Esperado ';' ao final da declaração")

    def stmt(self):  
        """Analisa diferentes tipos de comandos baseado no primeiro token"""
        t = self.peek().type
        
        if t == "IF":
            self.if_stmt()
        elif t == "WHILE":
            self.while_stmt()
        elif t == "IDENT":
            # Precisa olhar à frente para decidir que tipo de comando é
            if (self.look_type(1) == "LPAREN"
                and self.look_type(2) == "RPAREN"
                and self.look_type(3) == "LBRACE"):
                # É uma declaração de função: nome() { ... }
                self.func_decl()
            elif self.look_type(1) == "ASSIGN":
                # É uma atribuição: nome = expressão;
                self.assign_stmt()
            elif self.look_type(1) == "LPAREN":
                # É uma chamada de função: nome(argumentos);
                self.call_stmt()
            else:
                tok = self.peek()
                raise ParseError(f"Comando iniciado por IDENT inválido em {tok.line}:{tok.col}")
        elif t == "LBRACE":
            # É um bloco aninhado
            self.block()
        else:
            tok = self.peek()
            raise ParseError(f"Declaração/Comando inválido em {tok.line}:{tok.col} (token={tok.type})")

    def func_decl(self):
        """Declaração de função sem parâmetros: IDENT '(' ')' block"""
        self.expect("IDENT", "Esperado nome da função")
        self.expect("LPAREN", "Esperado '(' após nome da função")
        self.expect("RPAREN", "Esperado ')' após parâmetros da função")
        self.block()  # Corpo da função

    def call_stmt(self):
        """Chamada de função: IDENT '(' [argumentos] ')' ';'"""
        self.expect("IDENT", "Esperado nome da função na chamada")
        self.expect("LPAREN", "Esperado '(' na chamada de função")
        
        # Argumentos são opcionais
        if self.peek().type != "RPAREN":
            self.expr()  # Primeiro argumento
            while self.match("COMMA"):  # Argumentos adicionais separados por vírgula
                self.expr()
                
        self.expect("RPAREN", "Esperado ')' ao final da chamada")
        self.expect("SEMI", "Esperado ';' ao final da chamada de função")

    def if_stmt(self):  
        """Comando condicional: if '(' expressão ')' comando [else comando]"""
        self.expect("IF", "Esperado 'if'")
        self.expect("LPAREN", "Esperado '(' após 'if'")
        self.expr()  # Condição
        self.expect("RPAREN", "Esperado ')' após condição")
        self.stmt()  # Comando do 'then'
        
        # Else é opcional
        if self.match("ELSE"):
            self.stmt()  # Comando do 'else'

    def while_stmt(self):  
        """Loop: while '(' expressão ')' comando"""
        self.expect("WHILE", "Esperado 'while'")
        self.expect("LPAREN", "Esperado '(' após 'while'")
        self.expr()  # Condição do loop
        self.expect("RPAREN", "Esperado ')' após condição")
        self.stmt()  # Corpo do loop

    def assign_stmt(self):  
        """Atribuição: identificador '=' expressão ';'"""
        self.expect("IDENT", "Esperado identificador no início da atribuição")
        self.expect("ASSIGN", "Esperado '=' em atribuição")
        self.expr()  # Valor a ser atribuído
        self.expect("SEMI", "Esperado ';' após atribuição")

    # Métodos para analisar expressões seguindo precedência de operadores
    # Cada nível de precedência tem seu próprio método

    def expr(self):
        """Expressão no nível mais alto (menor precedência)"""
        self.or_()
        
    def or_(self):
        """Operador lógico OR (||) - associativo à esquerda"""
        self.and_()
        while self.match("OROR"):
            self.and_()
            
    def and_(self):
        """Operador lógico AND (&&) - associativo à esquerda"""
        self.equality()
        while self.match("ANDAND"):
            self.equality()
            
    def equality(self):
        """Operadores de igualdade (== !=) - associativo à esquerda"""
        self.rel()
        while self.match("EQEQ", "NEQ"):
            self.rel()
            
    def rel(self):
        """Operadores relacionais (< > <= >=) - associativo à esquerda"""
        self.add()
        while self.match("LT", "GT", "LE", "GE"):
            self.add()
            
    def add(self):
        """Operadores aditivos (+ -) - associativo à esquerda"""
        self.mul()
        while self.match("PLUS", "MINUS"):
            self.mul()
            
    def mul(self):
        """Operadores multiplicativos (* / %) - associativo à esquerda"""
        self.unary()
        while self.match("STAR", "SLASH", "PERCENT"):
            self.unary()
            
    def unary(self):
        """Operadores unários (! -) - associativo à direita"""
        if self.match("BANG", "MINUS"):
            self.unary()  # Recursão à direita para associatividade correta
        else:
            self.primary()
            
    def primary(self):
        """Expressões primárias (números, booleanos, identificadores, parênteses)"""
        if self.match("NUMBER", "TRUE", "FALSE", "IDENT"):
            return
        if self.match("LPAREN"):
            self.expr()  # Expressão entre parênteses
            self.expect("RPAREN", "Esperado ')' para fechar expressão")
            return
        t = self.peek()
        raise ParseError(f"Expressão inválida em {t.line}:{t.col} (token={t.type})")

**DEMO: código-fonte de teste**

In [ ]:
# Código fonte de exemplo para testar nosso analisador léxico e sintático
# Contém diversos elementos da linguagem: declarações, atribuições, 
# condicionais, loops, funções e expressões aritméticas
DEMO_SOURCE = r"""
main() {
  int x = 10;
  bool ok = true;
  int y;
  y = x + 20 * (3 - 1);

  if (y >= 50 && ok) {
    print(y);
  } else {
    y = y - 1;
  }

soma() {
  int a = 1;
  int b = 2;
  int c = a + b;
}
  while (y > 0) {
    y = y - 7;
  }
}

"""

In [ ]:
def print_lexeme_table(tokens: List[Token]):
    """Exibe uma tabela formatada com todos os tokens encontrados"""
    print("=" * 60)  
    print("TABELA DE LEXEMAS E TOKENS")  
    print("=" * 60)  
    print(f"{'Linha':<5} {'Col':<3} {'Token':<12} Lexema")  
    print("-" * 60)  
    
    # Percorre todos os tokens e os exibe em formato tabular
    for i, t in enumerate(tokens):
        print(f"{t.line:<5} {t.col:<3} {t.type:<12} {t.lexeme}")
        
        # Para listas muito grandes, mostra progresso a cada 20 tokens
        if (i + 1) % 20 == 0 and i + 1 < len(tokens):
            print(f"... ({i + 1}/{len(tokens)} tokens exibidos até agora)")
    
    print("-" * 60)  
    print(f"Total de tokens: {len(tokens)}")

def main():
    """Função principal que demonstra o funcionamento do analisador"""
    print("=" * 60)  
    print("ANALISADOR LÉXICO E PARSER - MINI LANGUAGE")  
    print("=" * 60)  

    # Fase 1: Análise Léxica
    lexer = Lexer(DEMO_SOURCE)  # Cria o analisador léxico
    tokens = lexer.tokenize()   # Extrai todos os tokens do código fonte

    # Verifica se o lexer funcionou corretamente
    assert tokens and tokens[-1].type == "EOF", "O lexer deve terminar com EOF"

    # Exibe a tabela de tokens na tela
    print_lexeme_table(tokens)

    # Cria e exibe a cadeia de tokens (útil para debug)
    chain_with_eof = " ".join(t.type for t in tokens)
    print("\nCADEIA DE TOKENS (inclui EOF):")
    print(chain_with_eof)

    # Salva os resultados em arquivos para análise posterior
    with open("tabela_tokens.txt", "w", encoding="utf-8") as f:
        f.write("=" * 60 + "\n")
        f.write("TABELA DE LEXEMAS E TOKENS\n")
        f.write("=" * 60 + "\n")
        f.write(f"{'Linha':<5} {'Col':<3} {'Token':<12} Lexema\n")
        f.write("-" * 60 + "\n")
        for t in tokens:
            f.write(f"{t.line:<5} {t.col:<3} {t.type:<12} {t.lexeme}\n")
        f.write("-" * 60 + "\n")
        f.write(f"Total de tokens: {len(tokens)}\n")
        
    with open("cadeia_tokens.txt", "w", encoding="utf-8") as f:
        f.write(chain_with_eof + "\n")
    print("\n[OK] Arquivos salvos: tabela_tokens.txt, cadeia_tokens.txt")

    # Fase 2: Análise Sintática
    print("\nFASE SINTÁTICA (versão inicial):")
    parser = Parser(tokens)  # Cria o analisador sintático
    try:
        parser.parse()  # Tenta analisar a sequência de tokens
        print("✅ Parse concluído sem erros (programa reconhecido).")  
    except ParseError as e:
        print(f"❌ Erro de sintaxe: {e}")  
    except SyntaxError as e:
        print(f"❌ Erro léxico (inesperado): {e}")

In [ ]:
# Importa sys para controlar a saída de forma mais precisa
import sys

def print_lexeme_table(tokens: List[Token]):
    """Versão alternativa da função que exibe tokens com formatação ligeiramente diferente"""
    sys.stdout.write("=" * 72 + "\n")
    sys.stdout.write("TABELA DE LEXEMAS E TOKENS\n")
    sys.stdout.write("=" * 72 + "\n")
    sys.stdout.write(f"{'Linha':<6} {'Col':<4} {'Token':<12} Lexema\n")
    sys.stdout.write("-" * 72 + "\n")
    
    # Exibe cada token usando sys.stdout.write para controle direto da saída
    for t in tokens:  
        sys.stdout.write(f"{t.line:<6} {t.col:<4} {t.type:<12} {t.lexeme}\n")
    sys.stdout.write("-" * 72 + "\n")
    sys.stdout.flush()  # Força a saída imediata

def main():
    """Versão simplificada da função principal para demonstração"""
    print("=" * 72)
    print("ANALISADOR LÉXICO E PARSER - MINI LANGUAGE")
    print("=" * 72)

    # Executa apenas a análise léxica
    lexer = Lexer(DEMO_SOURCE)
    tokens = lexer.tokenize()

    # Verifica integridade dos tokens
    assert tokens and tokens[-1].type == "EOF", "O lexer deve terminar com EOF"

    # Exibe resultados
    print_lexeme_table(tokens)

    print("\nCADEIA DE TOKENS (inclui EOF):")
    print(" ".join(t.type for t in tokens))

    # Executa análise sintática
    print("\nFASE SINTÁTICA (versão inicial):")
    parser = Parser(tokens)
    try:
        parser.parse()
        print("✅ Parse concluído sem erros (programa reconhecido).")
    except ParseError as e:
        print(f"❌ Erro de sintaxe: {e}")
    except SyntaxError as e:
        print(f"❌ Erro léxico (inesperado): {e}")

In [ ]:
# Ponto de entrada do programa - executa a função main quando o script é rodado diretamente
if __name__ == "__main__":
    main()

ANALISADOR LÉXICO E PARSER - MINI LANGUAGE
TABELA DE LEXEMAS E TOKENS
Linha  Col  Token        Lexema
------------------------------------------------------------------------
3      1    IDENT        main
3      5    LPAREN       (
3      6    RPAREN       )
3      8    LBRACE       {
4      3    INT          int
4      7    IDENT        x
4      9    ASSIGN       =
4      11   NUMBER       10
4      13   SEMI         ;
5      3    BOOL         bool
5      8    IDENT        ok
5      11   ASSIGN       =
5      13   TRUE         true
5      17   SEMI         ;
6      3    INT          int
6      7    IDENT        y
6      8    SEMI         ;
7      3    IDENT        y
7      5    ASSIGN       =
7      7    IDENT        x
7      9    PLUS         +
7      11   NUMBER       20
7      14   STAR         *
7      16   LPAREN       (
7      17   NUMBER       3
7      19   MINUS        -
7      21   NUMBER       1
7      22   RPAREN       )
7      23   SEMI         ;
10     1    IDENT        s

In [ ]:
@dataclass
class ASTNode:
    """
    Representa um nó na Árvore de Sintaxe Abstrata (AST).
    Cada nó tem um rótulo que descreve o que representa
    e pode ter filhos que são outros nós da árvore.
    """
    label: str                              # Nome/tipo do nó (ex: "If", "Assign", "Num(5)")
    children: List["ASTNode"] = field(default_factory=list)  # Lista de nós filhos
    
    def add(self, *nodes):
        """Adiciona um ou mais nós como filhos deste nó"""
        for n in nodes:
            if n is not None:  # Só adiciona nós válidos
                self.children.append(n)
        return self  # Retorna self para permitir encadeamento

def print_ast_ascii(node: ASTNode, prefix: str = "", is_last: bool = True):
    """
    Exibe a árvore AST em formato ASCII art, facilitando a visualização
    da estrutura hierárquica do programa analisado
    """
    # Escolhe o símbolo correto baseado se é o último filho ou não
    elbow = "└── " if is_last else "├── "
    print(prefix + elbow + node.label)
    
    # Calcula o prefixo para os filhos baseado na posição atual
    child_prefix = prefix + ("    " if is_last else "│   ")
    
    # Recursivamente imprime todos os filhos
    for i, ch in enumerate(node.children):
        print_ast_ascii(ch, child_prefix, i == len(node.children) - 1)

class ParserAST:
    """
    Versão do parser que constrói uma Árvore de Sintaxe Abstrata (AST)
    em vez de apenas validar a sintaxe. Cada método retorna um nó AST
    que representa a estrutura sintática encontrada.
    
    A AST é útil para:
    - Visualizar a estrutura do programa
    - Implementar interpretadores
    - Fazer análise semântica
    - Gerar código
    """
    def __init__(self, tokens: List["Token"]):
        self.tokens = tokens
        self.pos = 0

    def peek(self) -> "Token":
        """Olha o token atual sem avançar"""
        return self.tokens[self.pos]

    def advance(self) -> "Token":
        """Consome e retorna o token atual"""
        tok = self.tokens[self.pos]
        self.pos += 1
        return tok

    def match(self, *types) -> Optional["Token"]:
        """Verifica se o token atual é de um dos tipos dados e o consome se for"""
        if self.peek().type in types:
            return self.advance()
        return None

    def expect(self, ttype: str, msg: str) -> "Token":
        """Exige que o token atual seja de um tipo específico"""
        if self.peek().type == ttype:
            return self.advance()
        t = self.peek()
        raise ParseError(f"{msg} (encontrado {t.type} '{t.lexeme}' em {t.line}:{t.col})")

    def look_type(self, k: int) -> str:
        """Olha o tipo do token k posições à frente"""
        idx = self.pos + k
        return self.tokens[idx].type if idx < len(self.tokens) else "EOF"

    def parse(self) -> ASTNode:
        """Método principal que analisa o programa e retorna a raiz da AST"""
        # Verifica se começamos com 'main'
        if not (self.peek().type == "IDENT" and self.peek().lexeme == "main"):
            t = self.peek()
            raise ParseError(f"Esperado identificador 'main' (encontrado {t.type} '{t.lexeme}' em {t.line}:{t.col})")
        
        self.advance()  # Consome 'main'
        self.expect("LPAREN", "Esperado '(' após 'main'")
        self.expect("RPAREN", "Esperado ')' após '('")
        
        # Analisa o bloco principal
        main_block = self.block()
        self.expect("EOF", "Esperado fim do arquivo")

        # Cria e retorna a raiz da AST
        root = ASTNode("Program").add(ASTNode("Func(main)").add(main_block))
        return root

    def block(self) -> ASTNode:
        """Analisa um bloco e retorna nó AST correspondente"""
        self.expect("LBRACE", "Esperado '{'")
        
        # Coleta todas as declarações
        decls = ASTNode("DeclList")
        while self.peek().type in ("INT", "BOOL"):
            decls.add(self.decl())
            
        # Coleta todos os comandos
        stmts = ASTNode("StmtList")
        while self.peek().type in ("IF", "WHILE", "IDENT", "LBRACE"):
            stmts.add(self.stmt())
            
        self.expect("RBRACE", "Esperado '}'")
        return ASTNode("Block").add(decls, stmts)

    def decl(self) -> ASTNode:
        """Analisa uma declaração de variável"""
        typ = self.advance()  # Consome o tipo (INT ou BOOL)
        ident = self.expect("IDENT", "Esperado identificador")
        
        # Cria nó da declaração
        node = ASTNode("Decl").add(ASTNode(typ.type.lower()), ASTNode(f"Id({ident.lexeme})"))
        
        # Verifica se há inicialização
        if self.match("ASSIGN"):
            node.add(ASTNode("Init").add(self.expr()))
            
        self.expect("SEMI", "Esperado ';'")
        return node

    def stmt(self) -> ASTNode:
        """Analisa um comando e retorna nó AST apropriado"""
        t = self.peek().type
        
        if t == "IF":
            return self.if_stmt()
        if t == "WHILE":
            return self.while_stmt()
        if t == "LBRACE":
            return self.block()
        if t == "IDENT":
            # Decide que tipo de comando é baseado no lookahead
            if (self.look_type(1) == "LPAREN"
                and self.look_type(2) == "RPAREN"
                and self.look_type(3) == "LBRACE"):
                return self.func_decl()  # Declaração de função
            if self.look_type(1) == "ASSIGN":
                return self.assign_stmt()  # Atribuição
            if self.look_type(1) == "LPAREN":
                return self.call_stmt()  # Chamada de função
                
        tok = self.peek()
        raise ParseError(f"Comando inválido em {tok.line}:{tok.col} (token={tok.type})")

    def func_decl(self) -> ASTNode:
        """Analisa declaração de função"""
        name_tok = self.expect("IDENT", "Esperado nome da função")
        self.expect("LPAREN", "Esperado '(' após nome da função")
        self.expect("RPAREN", "Esperado ')' após parâmetros da função")
        body = self.block()
        return ASTNode(f"Func({name_tok.lexeme})").add(body)

    def call_stmt(self) -> ASTNode:
        """Analisa chamada de função"""
        name_tok = self.expect("IDENT", "Esperado nome da função na chamada")
        self.expect("LPAREN", "Esperado '(' na chamada de função")
        
        # Coleta argumentos
        args = ASTNode("Args")
        if self.peek().type != "RPAREN":
            args.add(self.expr())
            while self.match("COMMA"):
                args.add(self.expr())
                
        self.expect("RPAREN", "Esperado ')' ao final da chamada")
        self.expect("SEMI", "Esperado ';' ao final da chamada de função")
        return ASTNode(f"Call({name_tok.lexeme})").add(args)

    def if_stmt(self) -> ASTNode:
        """Analisa comando if/else"""
        self.expect("IF", "Esperado 'if'")
        self.expect("LPAREN", "Esperado '('")
        cond = self.expr()  # Condição
        self.expect("RPAREN", "Esperado ')'")
        thenp = self.stmt()  # Comando do then
        
        # Verifica se há else
        if self.match("ELSE"):
            elsep = self.stmt()
            return ASTNode("IfElse").add(cond, thenp, elsep)
        return ASTNode("If").add(cond, thenp)

    def while_stmt(self) -> ASTNode:
        """Analisa comando while"""
        self.expect("WHILE", "Esperado 'while'")
        self.expect("LPAREN", "Esperado '('")
        cond = self.expr()  # Condição
        self.expect("RPAREN", "Esperado ')'")
        body = self.stmt()  # Corpo do loop
        return ASTNode("While").add(cond, body)

    def assign_stmt(self) -> ASTNode:
        """Analisa comando de atribuição"""
        ident = self.expect("IDENT", "Esperado identificador")
        self.expect("ASSIGN", "Esperado '='")
        e = self.expr()  # Expressão do lado direito
        self.expect("SEMI", "Esperado ';' após atribuição")
        return ASTNode("Assign").add(ASTNode(f"Id({ident.lexeme})"), e)

    def expr(self) -> ASTNode:
        """Ponto de entrada para análise de expressões"""
        return self.or_()

    def or_(self) -> ASTNode:
        """Operador lógico OR - menor precedência"""
        node = self.and_()
        while self.match("OROR"):
            node = ASTNode("||").add(node, self.and_())
        return node

    def and_(self) -> ASTNode:
        """Operador lógico AND"""
        node = self.equality()
        while self.match("ANDAND"):
            node = ASTNode("&&").add(node, self.equality())
        return node

    def equality(self) -> ASTNode:
        """Operadores de igualdade"""
        node = self.rel()
        while True:
            if self.match("EQEQ"):
                node = ASTNode("==").add(node, self.rel())
            elif self.match("NEQ"):
                node = ASTNode("!=").add(node, self.rel())
            else:
                break
        return node

    def rel(self) -> ASTNode:
        """Operadores relacionais"""
        node = self.add()
        while True:
            if self.match("LT"):
                node = ASTNode("<").add(node, self.add())
            elif self.match("GT"):
                node = ASTNode(">").add(node, self.add())
            elif self.match("LE"):
                node = ASTNode("<=").add(node, self.add())
            elif self.match("GE"):
                node = ASTNode(">=").add(node, self.add())
            else:
                break
        return node

    def add(self) -> ASTNode:
        """Operadores aditivos"""
        node = self.mul()
        while True:
            if self.match("PLUS"):
                node = ASTNode("+").add(node, self.mul())
            elif self.match("MINUS"):
                node = ASTNode("-").add(node, self.mul())
            else:
                break
        return node

    def mul(self) -> ASTNode:
        """Operadores multiplicativos"""
        node = self.unary()
        while True:
            if self.match("STAR"):
                node = ASTNode("*").add(node, self.unary())
            elif self.match("SLASH"):
                node = ASTNode("/").add(node, self.unary())
            elif self.match("PERCENT"):
                node = ASTNode("%").add(node, self.unary())
            else:
                break
        return node

    def unary(self) -> ASTNode:
        """Operadores unários - maior precedência"""
        if self.match("BANG"):
            return ASTNode("!").add(self.unary())
        if self.match("MINUS"):
            return ASTNode("neg").add(self.unary())  # Negativo unário
        return self.primary()

    def primary(self) -> ASTNode:
        """Expressões primárias - folhas da árvore"""
        if (tok := self.match("NUMBER")):
            return ASTNode(f"Num({tok.lexeme})")
        if (tok := self.match("TRUE")):
            return ASTNode("Bool(true)")
        if (tok := self.match("FALSE")):
            return ASTNode("Bool(false)")
        if (tok := self.match("IDENT")):
            return ASTNode(f"Id({tok.lexeme})")
        if self.match("LPAREN"):
            e = self.expr()  # Expressão entre parênteses
            self.expect("RPAREN", "Esperado ')'")
            return ASTNode("( )").add(e)  # Nó para indicar parênteses
            
        t = self.peek()
        raise ParseError(f"Expressão inválida em {t.line}:{t.col} (token={t.type})")

# Demonstração: cria AST do código de exemplo e a exibe
tokens_ast = Lexer(DEMO_SOURCE).tokenize()  # Tokeniza o código
ast_root = ParserAST(tokens_ast).parse()    # Constrói a AST
print("ÁRVORE DE SINTAXE (ASCII):")
print_ast_ascii(ast_root)  # Exibe a árvore em formato visual

ÁRVORE DE SINTAXE (ASCII):
└── Program
    └── Func(main)
        └── Block
            ├── DeclList
            │   ├── Decl
            │   │   ├── int
            │   │   ├── Id(x)
            │   │   └── Init
            │   │       └── Num(10)
            │   ├── Decl
            │   │   ├── bool
            │   │   ├── Id(ok)
            │   │   └── Init
            │   │       └── Bool(true)
            │   └── Decl
            │       ├── int
            │       └── Id(y)
            └── StmtList
                ├── Assign
                │   ├── Id(y)
                │   └── +
                │       ├── Id(x)
                │       └── *
                │           ├── Num(20)
                │           └── ( )
                │               └── -
                │                   ├── Num(3)
                │                   └── Num(1)
                ├── IfElse
                │   ├── &&
                │   │   ├── >=
                │   │   │   ├── Id(y)
            